In [10]:
import ipytest
ipytest.autoconfig()

In [ ]:
def parseDICOMAge(age_str: str):
    import re
    extracted = re.match(r"(\d{3})([DWMY])", age_str)
    if extracted is not None:
        value = str(extracted[1])
        match (extracted[2]):
            case 'D':
                return f"{int(value)} day(s)"
            case 'W':
                return f"{int(value)} week(s)"
            case 'M':
                return f"{int(value)} month(s)"
            case 'Y':
                return f"{int(value)} year(s)"
            case _:
                raise Exception("I should never end up in this case!")
    return "Date format invalid"

In [11]:
%%ipytest

def test_parseDICOMAge():
    invalid = "00M"
    invalid_time = "000R"
    valid_d = "032D"
    valid_w = "041W"
    valid_m = "001M"
    invalid_m = "001m"
    valid_y = "043Y"

    assert parseDICOMAge(invalid) == "Date format invalid"
    assert parseDICOMAge(invalid_time) == "Date format invalid"
    assert parseDICOMAge(valid_d) == "32 day(s)"
    assert parseDICOMAge(valid_w) == "41 week(s)"
    assert parseDICOMAge(valid_m) == "1 month(s)"
    assert parseDICOMAge(invalid_m) == "Date format invalid"
    assert parseDICOMAge(valid_y) == "43 year(s)"

test_parseDICOMAge()

.                                                                                            [100%]
1 passed in 0.01s


In [ ]:
def parseDICOMTime(time_str: str):
    from datetime import datetime
    import re
    try:
        match (len(time_str)):
            case 2:
                try:
                    return datetime.strptime(time_str, "%H").strftime("%H:00:00")
                except ValueError as e:
                    #self.logger.warning(f"Invalid hour (00-23): {e}")
                    return "Invalid"
            case 4:
                try:
                    return datetime.strptime(time_str, "%H%M").strftime("%H:%M:00")
                except ValueError as e:
                    #self.logger.warning(f"Invalid hour (00-23) or minutes (00-59): {e}")
                    return "Invalid"
            case 6:
                try:
                    return datetime.strptime(time_str, "%H%M%S").strftime("%H:%M:%S")
                except ValueError as e:
                    #self.logger.warning(f"Invalid hour (00-23),minutes (00-59) or seconds (00-59): {e}")
                    return "Invalid"
            case n if n in range(8,13+1):
                matches = re.match(r"(\d{6})\.(\d{1,6})", time_str)
                if matches is not None:
                    try:
                        return datetime.strptime(matches[1] + matches[2].ljust(6,"0"), "%H%M%S%f").strftime("%H:%M:%S.%f")
                    except ValueError as e:
                        #self.logger.warning(f"Invalid millisecond format: {e}")
                        return "Invalid"
                else:
                    return "Invalid"
            case _:
                    return "Invalid time format"
    except ValueError:
        return "Unknown"

'04:05:34.100000'

In [31]:
%%ipytest

def test_parseDICOMTime():
    invalid = "00M"
    valid_h = "05"
    invalid_h = "42"
    valid_hm = "0342"
    invalid_hm = "0498"
    invalid_hm2 = "3230"
    valid_hms = "040534"
    invalid_hms = "069230"
    invalid_hms2 = "035089"
    valid_hmsf = "040534.1"
    valid_hmsf2 = "040534.12"
    valid_hmsf3 = "040534.123"
    valid_hmsf4 = "040534.1234"
    valid_hmsf5 = "040534.12345"
    valid_hmsf6 = "040534.123456"
    invalid_hmsf = "040534.123456213"
    invalid_hmsf2 = "049934.123456"

    assert parseDICOMTime(invalid) == "Invalid time format"
    assert parseDICOMTime(valid_h) == "05:00:00"
    assert parseDICOMTime(invalid_h) == "Invalid"
    assert parseDICOMTime(valid_hm) == "03:42:00"
    assert parseDICOMTime(invalid_hm) == "Invalid"
    assert parseDICOMTime(invalid_hm2) == "Invalid"
    assert parseDICOMTime(valid_hms) == "04:05:34"
    assert parseDICOMTime(invalid_hms) == "Invalid"
    assert parseDICOMTime(invalid_hms2) == "Invalid"
    assert parseDICOMTime(valid_hmsf) == "04:05:34.100000"
    assert parseDICOMTime(valid_hmsf2) == "04:05:34.120000"
    assert parseDICOMTime(valid_hmsf3) == "04:05:34.123000"
    assert parseDICOMTime(valid_hmsf4) == "04:05:34.123400"
    assert parseDICOMTime(valid_hmsf5) == "04:05:34.123450"
    assert parseDICOMTime(valid_hmsf6) == "04:05:34.123456"
    assert parseDICOMTime(invalid_hmsf) == "Invalid time format"
    assert parseDICOMTime(invalid_hmsf2) == "Invalid"

test_parseDICOMTime()

.                                                                                            [100%]
1 passed in 0.01s
